# 概述

##  散列函数
散列函数，也称作哈希函数，消息摘要函数，单向函数或者杂凑函数。散列函数主要用于验证数据的完整性。
通过散列函数，可以创建消息的“数字指纹”，消息接收方可以通过校验消息的哈希值来验证消息的完整性，防止消息被篡改。散列函数具有以下特性：

1. 散列函数的运算过程是不可逆的，这个称为散列函数的单向性。
2. 对于一个已知的消息及其散列值，要找到另外一个消息使其获得相同的散列值是不可能的，这个特性称为散列函数的弱碰撞性。
    这个特性可以用来防止消息伪造。
3. 任意两个不同消息的散列值一定不同。
4. 对原始消息长度没有限制。

任何消息经过散列函数处理后，都会产生一个唯一的散列值，这个散列值可以用来验证消息的完整性。
计算消息散列值的过程被称为“消息摘要”，计算消息散列值的算法被称为消息摘要算法。
常使用的消息摘要算法有：MD—消息摘要算法，SHA—安全散列算法，MAC—消息认证码算法。本文主要来了解MD算法。

## MD5算法
MD5算法是典型的消息摘要算法，它是由MD4，MD3和MD2算法演变而来。无论是哪一种MD算法，其原理都是接受一个任意长度的消息并产生一个128位的消息摘要。
如果把得到的消息摘要转换成十六进制字符串，则会得到一个32字节长度的字符串，我们平常见到的大部分MD数字指纹就是一个长度为32的十六进制字符串。


## MD5算法原理 
假设原始消息长度是b（以bit为单位），注意这里b可以是任意长度，并不一定要是8的整数倍。计算该消息MD5值的过程如下：

### 步骤1：填充信息
在计算消息的MD5值之前，首先对原始信息进行填充，这里的信息填充分为两步。


    第一小步，对原始信息进行填充，填充之后，要求信息的长度对512取余等于448。填充的规则如下：假设原始信息长度为b bit，那么在信息的b+1 bit位填充1，剩余的位填充0，直到信息长度对512取余为448。这里有一点需要注意，如果原始信息长度对512取余正好等于448，这种情况仍然要进行填充，很明显，在这时我们要填充的信息长度是512位，直到信息长度对512取余再次等于448。所以，填充的位数最少为1，最大为512。

In [1]:
# 将字符串转变为bit数组
from bitarray import bitarray

data = "abcde"
bit_array = bitarray(endian="big")
bit_array.frombytes(data.encode("utf-8"))

bit_array.append(1)
while len(bit_array) % 512 != 448:
    bit_array.append(0)

In [2]:
# 考虑到后续的算法都是以小端进行运算，在此先转为小端
bit_array = bitarray(bit_array, endian="little")
print("填充后的数据为：", bit_array)

填充后的数据为： bitarray('0110000101100010011000110110010001100101100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')


    第二小步，填充信息长度，我们需要把原始信息长度转换成以bit为单位，然后在第一步操作的结果后面填充64bit的数据表示原始信息长度。第一步对原始信息进行填充之后，信息长度对512取余结果为448，这里再填充64bit的长度信息，整个信息恰好可以被512整除。其实从后续过程可以看到，计算MD5时，是将信息分为若干个分组进行处理的，每个信息分组的长度是512bit。

In [3]:
import struct
length = (len(data) * 8) % pow(2, 64)
length_bit_array = bitarray(endian="little")
length_bit_array.frombytes(struct.pack("<Q", length))

result = bit_array.copy()
result.extend(length_bit_array)
print("填充长度后的数据为：", result)

填充长度后的数据为： bitarray('01100001011000100110001101100100011001011000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001010000000000000000000000000000000000000000000000000000000000')


### 2.信息处理
* 息分组定义
在进行MD5值计算之前，我们先来做一些定义。

```
Mp 代表经过填充之后的信息
LM 表示Mp的长度（以bit为单位）
N 表示分组个数，N = LM/512
M[i] 表示将原始信息进行分组后的第i个信息分组，其中i=1...N
X[i] 表示将M[i]进行分组后的第i个小分组，其中i=1...16
```

* 幻数定义  
现定义四个标准幻数如下:
```
AA = 01 23 45 67
BB = 89 ab cd ef
CC = fe dc ba 98
DD = 76 54 32 10
```
在计算机中存储时，采用小端存储方式，以AA为例，AA在Java中初始化的代码为为AA=0x67452301

In [4]:
AA=0x67452301
BB=0xefcdab89
CC=0x98badcfe
DD=0x10325476

In [5]:
# 表T, 是一个常量表，T[i] = 4294967296 * abs(sin(i))的运算结果取整，其中i=1...64
# Compute the T table from the sine function. Note that the
# RFC starts at index 1, but we start at index 0.
from math import floor as floor
from math import sin as sin
T = [floor(pow(2, 32) * abs(sin(i + 1))) for i in range(64)]

In [6]:
# 定义四个辅助方法。
'''
F(x,y,z) = (x & y) | ((~x) & z)
G(x,y,z) = (x & z) | (y & (~z))
H(x,y,z) = x ^ y ^ z
I(x,y,z) = y ^ (x | (~z))
其中，x，y，z长度为32bit
'''
F = lambda x, y, z: (x &y) | (~x &z)
G = lambda x, y, z: (x & z) | (y & ~z)
H = lambda x, y, z: x ^ y ^ z
I = lambda x, y, z: y ^ (x | ~z)

In [7]:
# 模运算函数（用于将幻数对 输入的消息进行变换）
modular_add = lambda a, b: (a + b) % pow(2, 32)

In [8]:
# 原始信息经过填充之后，最终得到的信息长度（bit）是512的整数倍，
# 我们先对信息进行分组，每512bit为一个分组，然后再将每个信息分组（512bit）再细分为16个小的分组，每个小分组的长度为32bit。
N = len(result) // 32

# 最终将输入到数据变换为4个int
buff_A = AA
buff_B = BB
buff_C = CC
buff_D = DD
    
# 依次处理每个分组，其中N代表分组个数, 每个分组的大小为 512（注意此前已经 //32 所以此时只需要//16）
for chunk_index in range(N//16):
    # 将块分解为16的小组， Break the chunk into 16 words of 32 bits in list X.
    start = chunk_index * 512
    X = [result[start + (x*32) : start + (x*32) + 32] for x in range(16)]
    
    # 将每个长度为32 bit的小组转为整数
    X = [int.from_bytes(word.tobytes(), byteorder="little") for word in X]
    
    A = buff_A
    B = buff_B
    C = buff_C
    D = buff_D
    
    # 以下做4轮不同的变换， 每轮做16此的位运算
    for i in range(4 * 16):
        if 0 <= i <= 15:
            k = i
            s = [7, 12, 17, 22]
            temp = F(B, C, D)
        elif 16 <= i <= 31:
            k = ((5 * i) + 1) % 16
            s = [5, 9, 14, 20]
            temp = G(B, C, D)
        elif 32 <= i <= 47:
            k = ((3 * i) + 5) % 16
            s = [4, 11, 16, 23]
            temp = H(B, C, D)
        elif 48 <= i <= 63:
            k = (7 * i) % 16
            s = [6, 10, 15, 21]
            temp = I(B, C, D)
            
        temp = modular_add(temp, X[k])
        temp = modular_add(temp, T[i])
        temp = modular_add(temp, A)
        temp = modular_add(temp, s[i % 4])
        temp = modular_add(temp, B)
        
        # Swap the registers for the next operation.
        A = D
        D = C
        C = B
        B = temp
        
        buff_A = modular_add(buff_A, A)
        buff_B = modular_add(buff_B, B)
        buff_C = modular_add(buff_C, C)
        buff_D = modular_add(buff_D, D)

### 3. 进行小端输出

In [9]:
a = struct.unpack("<I", struct.pack(">I", buff_A))[0]
b = struct.unpack("<I", struct.pack(">I", buff_B))[0]
c = struct.unpack("<I", struct.pack(">I", buff_C))[0]
d = struct.unpack("<I", struct.pack(">I", buff_D))[0]

# 按照消息的十六进制输出
hex = f"{format(A, '08x')}{format(B, '08x')}{format(C, '08x')}{format(D, '08x')}"
print("十六进制数值：", hex)

dex = A << (3*32) | B << (2*32) | C << 32 | D
print("十进制数值：", dex)

十六进制数值： 7b89b4382887a70ae715950b4c8a6153
十进制数值： 164210043434213076066579398883734610259


# 参考
https://github.com/wilsonloo/md5.py.git